In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.0
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)
matplotlib 3.3.4
numpy 1.19.5
pandas 1.3.4
sklearn 0.22.1
tensorflow 2.6.0
keras.api._v2.keras 2.6.0


In [27]:
np.random.choice(5,50)

array([0, 3, 2, 2, 2, 3, 4, 2, 0, 0, 1, 1, 0, 4, 0, 1, 1, 3, 2, 0, 3, 2,
       0, 3, 4, 1, 4, 0, 0, 3, 2, 2, 3, 2, 4, 1, 2, 2, 2, 0, 0, 4, 1, 0,
       1, 4, 3, 2, 2, 2])

In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [6]:
# 首先我们看下metric(算子)使用,算子使用了均方误差
metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print(metric([0.], [1.]))
#具有累加功能，第1个是9，第二个是1，平均是5，(9+1)/2
print(metric.result())
print('-'*50)
# #不想累加就reset，即重置为0，每次epoch计算一次平均
metric.reset_states()  #每次epoch需要reset
metric([1.], [3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(4.0, shape=(), dtype=float32)


In [7]:
#训练集的样本数
print(len(x_train_scaled))
print(x_train.shape[1:])  #特征数，shape的第一个是样本数，后面是各个特征的数

11610
(8,)


In [39]:
11610/32  #每次训练给予的样本数

362.8125

In [40]:
t= np.arange(6).reshape(1, 2, 1, 3)
print(t)
tf.squeeze(t)  # [2, 3]  #降维，将维数为1去除

[[[[0 1 2]]

  [[3 4 5]]]]


<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[0, 1, 2],
       [3, 4, 5]])>

In [10]:
#随机挑选5个样本，看下特征，标签
idx = np.random.randint(0, 1000, size=5)
print(idx)
print(x_train_scaled[idx])
y_train[idx]

[854 913  17 894 892]
[[ 1.87100283  0.27216142  0.55226273 -0.08799606 -0.55393901 -0.07631072
   1.09455191 -1.48058947]
 [ 1.15827914 -1.16964284  0.29999114 -0.03246526  0.38963862 -0.0464411
  -0.65974741  0.35201325]
 [-0.38966761 -1.16964284  0.18295192 -0.0098568  -1.02209868 -0.11612091
   1.6870945  -0.71159541]
 [ 0.77484915  0.35226166  0.09019054 -0.02088154 -0.24818356 -0.12980283
  -0.95368586  0.85136004]
 [ 0.04443525 -1.00944236 -0.04988441 -0.31892586  0.03126058  0.08654257
  -0.71573568  1.01614448]]


array([4.44   , 2.871  , 1.361  , 5.00001, 1.315  ])

In [42]:
# squeeze的作用
t=tf.constant([[1],[2],[3]])
print(t)
tf.squeeze(t,1) # 指定删除1轴

tf.Tensor(
[[1]
 [2]
 [3]], shape=(3, 1), dtype=int32)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [13]:
for i in range(10):
    print('helloworld',end='\r')  # 不断地在原地覆盖地打印

In [14]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch结束 验证集 metric

epochs = 100 #多少次
batch_size = 32 
steps_per_epoch = len(x_train_scaled) // batch_size  #每一个epoch拿多少数据
print(steps_per_epoch)
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

#随机取数据,取出来32个样本
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])
print(model.variables)
print('-'*50)

#下面一部分相当于替代了fit函数
# print(model.variables)#可以看下这个里边有什么
# model.summary()
for epoch in range(epochs):#每一轮epochs训练所有的样本
    metric.reset_states()  #清空损失
    for step in range(steps_per_epoch):
        #随机取32个样本
        x_batch, y_batch = random_batch(x_train_scaled, y_train,
                                        batch_size)
        with tf.GradientTape() as tape:
            #得到预测值
            y_pred = model(x_batch) #等价于model.predict
 
            #删减了值为1的维度,二阶张量，变为一阶张量
            y_pred = tf.squeeze(y_pred, 1)
            #计算损失
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            #计算均方误差
            metric(y_batch, y_pred)
        #求梯度
        grads = tape.gradient(loss, model.variables)
        #梯度和变量绑定
        grads_and_vars = zip(grads, model.variables)
        #更新，通过grads，去更新模型的model.variables，也就是更新了w,b
        optimizer.apply_gradients(grads_and_vars)
        #打印，不要在循环内加print，影响\r
        p="Epoch "+str(epoch)+" train mse:"+str(metric.result().numpy())
        print(p, end='\r')
    print('') #打换行的目的是为了新起一行显示
    #搞了一波训练后，认为模型可以了，去验证集验证
    y_valid_pred = model(x_valid_scaled)
    # 删减了值为1的维度
#     print(y_valid_pred.shape)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
#     print(y_valid_pred.shape)
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print("\t", "valid mse: ", valid_loss.numpy())
        


362
[<tf.Variable 'dense/kernel:0' shape=(8, 30) dtype=float32, numpy=
array([[-2.33394578e-01,  3.90441447e-01, -2.02911705e-01,
         3.72229904e-01,  4.80298698e-02,  3.34867150e-01,
         1.95421249e-01, -2.05333576e-01, -3.51379693e-01,
        -2.62964070e-02,  1.48886055e-01, -7.78056085e-02,
         3.43607217e-01,  1.38993651e-01, -1.84959233e-01,
         2.01822609e-01,  3.46111864e-01, -2.19839290e-01,
        -2.15609804e-01, -1.11134708e-01,  1.00734472e-01,
        -2.57328868e-01,  4.51035798e-02,  9.10943747e-03,
         2.45002806e-02,  3.14507216e-01,  7.00456202e-02,
         2.31188387e-01,  1.81470066e-01, -3.94834638e-01],
       [ 2.36808449e-01,  2.89643139e-01, -1.53902173e-03,
        -3.61920595e-02, -3.54213864e-01, -6.83941543e-02,
         3.90978605e-01, -2.01325879e-01, -1.56647757e-01,
        -2.84752578e-01,  3.28161985e-01, -2.03438625e-01,
        -1.79174259e-01, -2.89041370e-01,  3.40324789e-01,
         2.50680298e-01, -1.52389765e-01,  

Epoch 36 train mse:0.33469582
	 valid mse:  0.3569989720128385
Epoch 37 train mse:0.34318417
	 valid mse:  0.3626712218248998
Epoch 38 train mse:0.35200134
	 valid mse:  0.3559554156846957
Epoch 39 train mse:0.34155953
	 valid mse:  0.3530325844486809
Epoch 40 train mse:0.33059064
	 valid mse:  0.3494478273496873
Epoch 41 train mse:0.33510765
	 valid mse:  0.3496204292368414
Epoch 42 train mse:0.33524448
	 valid mse:  0.3503388826909692
Epoch 43 train mse:0.32299617
	 valid mse:  0.35068492162632314
Epoch 44 train mse:0.31961608
	 valid mse:  0.3587231824524005
Epoch 45 train mse:0.31461377
	 valid mse:  0.35214042838454296
Epoch 46 train mse:0.33954784
	 valid mse:  0.35313288104384577
Epoch 47 train mse:0.32969585
	 valid mse:  0.3465945578630811
Epoch 48 train mse:0.32155624
	 valid mse:  0.3462305711924993
Epoch 49 train mse:0.31947774
	 valid mse:  0.3460096748104059
Epoch 50 train mse:0.34188542
	 valid mse:  0.34332045085938984
Epoch 51 train mse:0.32595626
	 valid mse:  0.34909